In [35]:
! pip install fonttools
! pip install pascal-voc-writer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import random
import string
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from tqdm import tqdm
from io import BytesIO
from toolz.curried import *
import requests

In [3]:
from fontTools.ttLib import TTFont
from fontTools.unicode import Unicode

In [4]:
! git clone https://github.com/fatihkaan22/ocr-digital-display/

Cloning into 'ocr-digital-display'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 103 (delta 53), reused 82 (delta 41), pack-reused 0
Receiving objects: 100% (103/103), 2.08 MiB | 4.27 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [107]:
paths = {
  'fonts': os.path.join('ocr-digital-display','synthetic-data-generation', 'fonts'),
  'drive_ws': os.path.join('drive', 'MyDrive', 'cse496')
}

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def get_wordlist():
  word_site = "https://www.mit.edu/~ecprice/wordlist.10000"
  response = requests.get(word_site)
  return response.content.splitlines()

WORDS = get_wordlist()
# ALPHAS = [':', '.'] # TODO: &
ALPHAS = []

def char_list():
  return concatv(
      string.ascii_letters,
      string.digits,
      ALPHAS
  )

def supported_chars(ttfont):
  return pipe(
    ttfont,
    lambda f: f['cmap'].tables,
    map(lambda x: x.cmap.items()),
    merge,
    lambda d: list(d.keys()),
    filter(lambda char_code: chr(char_code) in char_list()),
    list
  )

def is_supported(charset, font):
  return set(
      map(ord)(charset)
    ).issubset(supported_chars(font))
    
def random_string_from_charset(charset):
  return pipe(
      range(random.randrange(5,15)),
      map(lambda _: random.choice(charset)),
      reduce(lambda a, b: str(a) + str(b)),
  )

def random_digits():
  return random_string_from_charset(string.digits)

def random_word():
  return pipe(
      WORDS,
      random.choice,
      lambda bs: bs.decode("utf-8") 
  )

# TODO: test curry
def supported_filter(font):
  def test(string):
    return is_supported(string, font)
  return test

def sample_text(font):
  return pipe([
#      random_string_from_charset(list(string.digits) + ALPHAS),
#      random_word(),
      random_digits()
    ],
    filter(supported_filter(font)),
    filter(supported_filter(font)),
    list,
    random.choice
  )

In [7]:
# TODO: unused
def is_digits_supported(font):
  return is_supported(string.digits, font)

# TODO: unused
def is_letters_supported(font):
  return is_supported(string.ascii_letters, font)

# TODO: unused
def is_alphas_supported(font):
  return is_supported(ALPHAS, font)

# TODO: unused
def random_gibberish(font):
  return pipe(
      range(random.randrange(5,15)),
      map(lambda _: chr(random.choice(supported_chars(font)))),
      reduce(lambda a, b: str(a) + str(b)),
  )

In [61]:
def draw_sample(font_path):
  word = sample_text(TTFont(font_path))
  font = ImageFont.truetype(font_path, random.randrange(20, 34))
  img = Image.new("RGBA", (512,512),(255,255,255))
  draw = ImageDraw.Draw(img)
  xy = (random.randrange(5, 100), 
        random.randrange(10, 400))
  draw.text(xy,word,(0,0,0),font=font)
  draw = ImageDraw.Draw(img)
  objects = []
  for i, char in enumerate(word):
    right, bottom = font.getsize(word[:i+1])
    width, height = font.getmask(char).size
    right += xy[0]
    bottom += xy[1]
    top = bottom - height
    left = right - width
#    draw.rectangle((left, top, right, bottom), None, '#f00')
    objects.append({
        'name': str(char),
        'xmin': left,
        'xmax': right,
        'ymin': top,
        'ymax': bottom
    })
  return {'img': img, 'text': word, 'font': font.getname(), 'objects': objects}

def generate_data(fonts, sample_count=1):
  return pipe(
      fonts,
      mapcat(lambda font: pipe(
          range(sample_count),
          map(lambda _: font),
          map(draw_sample),
      )),
  )

def get_font_path(filename):
  return os.path.join(paths['fonts'], filename)

font_paths = map(get_font_path)(os.listdir(paths['fonts']))

#for d in generate_data(font_paths, 1):
#  print(d['text'])
#  print(d['font'])
#  display(d['img'])

In [93]:
import xml.etree.cElementTree as ET
from xml.dom import minidom

def prettify(elem):
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def generate_xml_annotation(filename, im_width, im_height, bboxes):
    try:
        annotation = ET.Element('annotation')
        ET.SubElement(annotation, 'filename').text = filename
        size = ET.SubElement(annotation, 'size')
        ET.SubElement(size, 'width').text = str(im_width)
        ET.SubElement(size, 'height').text = str(im_height)
        ET.SubElement(size, 'depth').text = '3'

        for index, box in enumerate(bboxes):
            objectBox = ET.SubElement(annotation, 'object')
            ET.SubElement(objectBox, 'name').text = box['name']
            ET.SubElement(objectBox, 'pose').text = 'Unspecified'
            ET.SubElement(objectBox, 'truncated').text = '0'
            ET.SubElement(objectBox, 'difficult').text = '0'
            bndBox = ET.SubElement(objectBox, 'bndbox')
            ET.SubElement(bndBox, 'xmin').text = str(box['xmin'])
            ET.SubElement(bndBox, 'ymin').text = str(box['ymin'])
            ET.SubElement(bndBox, 'xmax').text = str(box['xmax'])
            ET.SubElement(bndBox, 'ymax').text = str(box['ymax'])

        et = ET.ElementTree(annotation)
        return prettify(annotation)
    except Exception as e:
        print('Error to generate the XML for image {}'.format(filename))
        print(e)

In [91]:
import uuid

def random_id():
  return str(uuid.uuid4())[:8]

In [114]:
dseg7_font = ('DSEG7 Classic', 'BoldItalic')

for i in range(500):
  for d in generate_data([get_font_path('DSEG7Classic-BoldItalic.ttf')]):
#    display(d['img'])
    print(d['text'])
    filename = random_id()
    path = os.path.join('.', paths['drive_ws'], 'dataset_v1', filename) # TODO: create this path in drive
    d['img'].save(f'{path}.png')
    xml_string = generate_xml_annotation(f'{filename}.png', 512, 512, d['objects'])
    with open(f'{path}.xml', 'w') as file:
      file.write(xml_string)

5627834230
4929030841547
49832844972046
6056843299727
63274267672
8032952938668
83596706007462
620283896
8383763
657577657
09716334350684
38145622978
66095641515
95658947
010230376
501960969
0741664
324267
64225
70131517
324266040
14599929156049
2379941267
710827562
5974219702431
02363
899149567
96021
2025202
65705
7188750282338
50943
6382364269
964308
23348995699141
1224378148
895185306
482817
31366103
243990
43747647081
027731389131
0980847788232
2427864302500
0612577572257
39719154
37598572
9389229
41503541
43407280
97180760084
25248016992256
6937352002152
15678903
85034
21288121335802
93171
19947228
4213819852
307200
289242
6971449741
979710647
52790285
664405273
2509110246323
5723986667529
6257061610703
76795972013518
91475049828
941512946220
204631346502
803073855
00662
12710443
694898390423
9017427
84135104
47083961
87535963100142
5886035495
45344103
56573899
4545702
93466316
1337803
1768000038129
0070563
1896078240671
42948326
17522178
05262059
951298
86246241344603
7448284104
